# 决策树

决策树是一种基本的分类与回归方法。

决策树在分类问题中表示基于特征对实例进行分类的过程，可以认为是 if-then 规则的集合，也可以认为定义在特征空间与类空间上的条件概率分布。

其主要优点是模型具有**可读性，分类速度快**。学习时利用训练数据，根据损失函数最小化的原则建立决策树模型。

决策树学习通常包括 3 个步骤:**特征选择，决策树生成和决策树的修剪**。

## 决策树模型

决策树由结点与有向边组成。结点分为内部结点与叶结点，内部结点表示一个特征或属性，叶结点表示一个类。


### if-then 规则

可以将决策树看成一个 if-then 规则的集合，由决策树的根节点到叶结点的每一条路径构建一条规则:路径上的内部结点对应着规则的条件，叶结点则是规则对应的结论。决策树路径具有有**互斥且完备**的性质，每一个实例均有且仅有一条路经覆盖。

### 条件概率分布


决策树还表示给定特征条件下类的条件概率分布，这一条件概率分布定义在特征空间的一个划分上，将特征空间划分为互不相交的单元或区域,并在每个单元定义一个类的概率分布就构成了一个条件概率分布，决策树的一条路径对应于划分中的一个单元，决策树所表示的条件概率分布由各个单元给定条件下类的条件概率分布组成。假设 X 为表示特征的随机变量，Y 为表示类的随机变量，那么这个条件概率分布可以表示为 P(Y|X)。X 取值于给定划分下单元的集合，Y 取值于类的集合。各叶结点上的条件概率往往偏向某一个类，即属于某一类的概率较大。 

## 决策树学习

决策树学习本质上是从训练数据集中归纳出一组分类规则，与训练数据集不相矛盾的决策树可能有多个，我们需要的是一个与训练数据矛盾较小的决策树，同时具有很好的泛化能力。

决策树学习用损失函数表示这一目标，通常是正则化的极大似然函数。决策树的学习策略是以损失函数为目标函数的最小化。

决策树学习的算法通常是一个递归地选择最优特征，并根据该特征对训练数据进行分割，使得对各个子数据集有一个最好的分类过程，这一分类过程对应着空间的划分，也对应着决策树的构建。

以上方法生成的决策树对训练数据可能有很好的分类能力，但可能发生过拟合现象。我们需要对已生成的树自下而上进行剪枝，使其具有更好的泛化能力。如果特征数量很多，也可以在决策树学习开始的时候对特征进行选择只留下对训练数据有足够分类能力的特征。

决策树算法包括特征选择、决策树生成和决策树的剪枝过程。决策树的生成对应于模型的局部选择，决策树的剪枝对应于模型的全局选择，决策树的生成只考虑局部最优。


## 特征选择

特征选择在于选取对训练数据具有分类能力的特征，以提高决策树学习效率。如果利用一个特征进行分类的结果与随机分类的结果没有很大差别，则称这个特征没有分类能力。通常特征选择的准则是信息增益或信息增益比。

### 信息增益

**熵(entropy)**:

表示随机变量 **不确定性** 的度量，设 X 是一个取有限个值的离散随机变量，其概率分布为

$$P(X=x_i)=p_i,i=1,2,...,n$$

则随机变量 X 的熵定义为

$$H(Y)=-\sum_{i=1}^n p_ilog p_i$$

通常对数以 2 或 e 为底，这时熵的单位分别为 bit 与 nat。

**条件熵**:

表示在已知随机变量 X 的条件下随机变量 Y 的不确定性。随机变量 X 给定的条件下随机变量 Y 的条件熵 H(Y|X)。定义为 X 给定条件下 Y 的条件概率分布的熵对 X 的数学期望

$$H(Y|X)=\sum_{i=1}^n p_iH(Y|X=x_i)$$

这里

$$p_i=P(X=x_i),i=1,2,...,n$$

当熵和条件熵中的概率由数据估计(特别是极大似然估计)得到时，所对应的熵与条件熵分别称为经验熵和经验条件熵。

**信息增益**:

特征 A 对训练数据集 D 的信息增益 g(D,A),定义为集合 D 的经验熵 H(D) 与特征 A 给定条件下 D 的经验条件熵 H(D|A) 之差

$$g(D,A)=H(D)-H(D|A)$$

**其表示得知特征 X 的信息而使得类 Y 的信息不确定性减少的程度。**

根据信息增益准则的特征选择方法是:对训练数据集 D,就算其每个特征的信息增益，并比较它们的大小，选择信息增益最大的特征。

**信息增益比**:

以信息增益作为划分训练数据集的特征，存在偏向选择取值较多特征的问题。如面对连续数据时，或数据间没有明显区分时，信息增益效果较差：

$$g(D,A)=H(D)-H(D|A)=H(D)-(-\sum^n_{n=1}\frac{D_i}{D}\sum^K_{k=1}\frac{D_{ik}}{D_i}log{\frac{D_{ik}}{D_i}})$$

当极限情况下，每个样本分到一个节点中去，即条件熵趋于0，此时信息增益达到最大，ID3 算法将选取此特征，然而此特征显然不一定为最佳选择。

使用信息增益比可以对这一问题进行校正。


$$g_R(D,A)=\frac{g(D,A)}{H_A(D)}$$


### 信息增益算法

设训练数据集为 D，|D| 表示其样本容量，即样本个数。设有 K 个类 C_k，|C_k| 为属于类 C_k 的样本个数

$$\sum^K_{k=1}|C_k|=|D|$$

设特征 A 有 n 个不同的取值，根据特征 A 的取值将 D 划分为 n 个子集 D_1,D_2,...,D_n，|D_i| 为 D_i 的样本个数

$$\sum^n_{i=1}|D_i|=|D|$$

记子集 Di 中属于类 C_k 的样本的集合为 D_ik

$$D_ik=D_i\bigcap C_k$$

|D_ik| 为 D_ik 的样本个数

**输入**:训练数据集 D 和 特征 A
**输出**:特征 A 对训练数据集 D 的信息增益 g(D,A)

* 计算数据集 D 的经验熵 H(D)

$$H(D)=-\sum^K_{k=1}\frac{|C_k|}{|D|} log_2 \frac{|C_k|}{|D|}$$

* 计算特征 A 对数据集 D 的经验条件熵 H(D)

$$H(D|A)=\sum^n_{i=1}{\frac{|D_i|}{|D|}H(D_i)}= -\sum^n_{i=1}{\frac{|D_i|}{|D|}\sum^K_{k=1}\frac{|D_ik|}{|Di|} log_2 \frac{|D_ik|}{|D_i|}}$$

* 计算信息增益

$$g(D,A)=H(D)-H(D|A)$$

In [9]:
import numpy as np
import math


def cal_info_gain(data, label):
    data_len = len(data)
    label_len = len(label)

    # init entropy
    result_map = {}
    for r in data[:, 4]:
        if r not in result_map:
            result_map[r] = 1
        else:
            times = result_map[r] + 1
            result_map[r] = times
    #print(result_map)

    entropy = 0
    for m in result_map:
        p = result_map[m] / data_len
        entropy += -p*math.log(p, 2)
    #print("entropy %f" % entropy)

    # conditional entropy
    gains = {}
    for l in range(label_len):
        #print()
        #print(label[l])

        # 每一特征的条件熵
        clz_map = {}
        i = 0
        for d in data[:, l]:
            if d not in clz_map:
                clz_map[d] = [i]
            else:
                clz_map[d].append(i)
            i += 1
        #print(clz_map)

        # H(Di)
        conditional_entropy = 0
        for m in clz_map:
            sub_conditional_entropy_map = {}
            for pos in clz_map[m]:
                if data[pos][4] not in sub_conditional_entropy_map:
                    sub_conditional_entropy_map[data[pos][4]] = 1
                else:
                    temp = sub_conditional_entropy_map[data[pos][4]] + 1
                    sub_conditional_entropy_map[data[pos][4]] = temp
            #print(sub_conditional_entropy_map)

            # H(Di)
            sub_conditional_entropy = 0
            for d in sub_conditional_entropy_map:
                p = sub_conditional_entropy_map[d] / len(clz_map[m])
                sub_conditional_entropy += -p * math.log(p, 2)

            p = len(clz_map[m]) / len(data)
            conditional_entropy += p * sub_conditional_entropy

        gains[label[l]] = entropy - conditional_entropy

    print(gains)
    print("最优特征：%s" % {max(gains, key=gains.get)})


def data_table():
    dataSet = np.array([[u'青年', u'否', u'否', u'一般', u'拒绝'],
               [u'青年', u'否', u'否', u'好', u'拒绝'],
               [u'青年', u'是', u'否', u'好', u'同意'],
               [u'青年', u'是', u'是', u'一般', u'同意'],
               [u'青年', u'否', u'否', u'一般', u'拒绝'],
               [u'中年', u'否', u'否', u'一般', u'拒绝'],
               [u'中年', u'否', u'否', u'好', u'拒绝'],
               [u'中年', u'是', u'是', u'好', u'同意'],
               [u'中年', u'否', u'是', u'非常好', u'同意'],
               [u'中年', u'否', u'是', u'非常好', u'同意'],
               [u'老年', u'否', u'是', u'非常好', u'同意'],
               [u'老年', u'否', u'是', u'好', u'同意'],
               [u'老年', u'是', u'否', u'好', u'同意'],
               [u'老年', u'是', u'否', u'非常好', u'同意'],
               [u'老年', u'否', u'否', u'一般', u'拒绝'],
               ])
    labels = [u'年龄', u'有工作', u'有房子', u'信贷情况']
    return dataSet, labels

data, label = data_table()
cal_info_gain(data, label)

{'年龄': 0.08300749985576883, '有工作': 0.32365019815155627, '有房子': 0.4199730940219749, '信贷情况': 0.36298956253708536}
最优特征：{'有房子'}


## 决策树的生成

### ID3 算法

ID 3 算法的核心是在决策树各个结点应用信息增益准则选择特征，递归构建决策树。

具体方法是从根节点开始，对结点计算所有可能的特征的信息增益，选择信息增益最大的特征作为结点的特征，由该特征的不同取值建立子结点，再对子结点递归调用以上方法，构建决策树，直到所有特征的信息增益均很小或没有特征可以选择为止。

**输入**:训练数据集 D，特征集 A，阙值 ε

**输出**:决策树 T

* 若 D 中所有实例属于同一类 C_k，则 T 为单节点树，并将类 C_k 作为该结点的类标记，返回 T
* 若 A 为 ∅，则 T 为单节点树，并将 D 中实例树最大的类 C_k 作为该结点的类标记，返回 T
* 否则计算 A 中各特征对 D 的信息增益，选择信息增益最大的特征 A_g
* 如果 A_g 的信息增益小于阙值 ε，则置 T 为单结点树，并将 D 中实例数量最大的类 C_k 作为该结点的类标记，返回 T
* 否则，对 A_g 的每一可能值 a_i，依 A_g=a_i 将 D 分割为若干非空子集 D_i，将 D_i 中实例数最大的类作为标记，构建子结点，由结点及其子结点构成树 T，返回 T
* 对第 i 个子结点，以 D_i 为训练集，以 A-{A_g} 为特征集，递归调用 1-5 步，得到子树 T_i，返回 T_i

In [15]:
def build_decision_tree(data, label, threshold=0.2):
    # data is empty
    if len(data) == 0:
        return {}

    feature, gain = cal_info_gain(data, label)
    feature_index = 0
    for i in range(0, len(label)):
        if label[i] == feature:
            feature_index = i
    if gain < threshold:
        return {}

    # same class
    clz_map = {}
    for d in data:
        if d[-1] not in clz_map:
            clz_map[d[-1]] = 1
        else:
            tmp = clz_map[d[-1]] + 1
            clz_map[d[-1]] = tmp
    if len(clz_map) < 2:
        for c in clz_map:
            return {c}

    sub_tree = {}
    tree = {feature: sub_tree}
    for d in data:
        if d[feature_index] not in sub_tree:
            f = d[feature_index]
            t = build_decision_tree(
                np.array([d for d in data if d[feature_index] == f]), label)
            if t == {}:
                t[max(clz_map, key=clz_map.get)] = {}
            sub_tree[d[feature_index]] = t

    return tree



data, label = data_table()
tree = build_decision_tree(data, label)
print(tree)

{'有房子': {'否': {'有工作': {'否': {'拒绝': {}}, '是': {'拒绝': {}}}}, '是': {'同意': {}}}}


### C4.5 算法

C4.5 算法与 ID3 算法相似，C4.5 算法对 ID3 算法进行了改进，在生成过程中使用**信息增益比**来选择特征

**输入**:训练数据集 D，特征集 A，阙值 ε

**输出**:决策树 T

* 若 D 中所有实例属于同一类 C_k，则 T 为单节点树，并将类 C_k 作为该结点的类标记，返回 T
* 若 A 为 ∅，则 T 为单节点树，并将 D 中实例树最大的类 C_k 作为该结点的类标记，返回 T
* 否则计算 A 中各特征对 D 的信息增益比，选择信息增益比最大的特征 A_g
* 如果 A_g 的信息增益小于阙值 ε，则置 T 为单结点树，并将 D 中实例数量最大的类 C_k 作为该结点的类标记，返回 T
* 否则，对 A_g 的每一可能值 a_i，依 A_g=a_i 将 D 分割为若干非空子集 D_i，将 D_i 中实例数最大的类作为标记，构建子结点，由结点及其子结点构成树 T，返回 T
* 对第 i 个子结点，以 D_i 为训练集，以 A-{A_g} 为特征集，递归调用 1-5 步，得到子树 T_i，返回 T_i

## 决策树的剪枝

决策树生成算法由于学习时过多考虑如何提高训练数据正确分类，易构建出过于复杂的决策树，出现过拟合现象。因此需要**剪枝**。
决策树剪枝往往通过极小化决策树整体的损失函数或代价函数来实现。

设树 T 的叶结点个数为 |T|，t是树 T 的叶结点，该叶结点有 N_i 个样本点，其中 k 类的样本点有 N_ik 个，H_i(T) 为叶结点上的经验熵，则决策树学习的损失函数可以定义为：

$$C_a(T) = \sum^{|T|}_{t=1}N_t H_t (T) + a|T|$$

其中经验熵为

$$H_t(T)=-\sum_k{\frac{N_{tk}}{N_t}}{log \frac{N_{tk}}{N_t}}$$

## CART 算法

CART(classification and regression tree) 即分类与回归树，其同样由特征选择，树生成及剪枝组成。既可用于分类也可用于回归。

CART 是在给定输入随机变量 x 条件下输出变量 y 的条件概率分布的学习方法。CART 假设决策树是二叉树，内部结点特征取值为"是"和"否"，左分支取值为"是"，右分支取值为"否",这样的决策树等价于递归二分每个特征，将输入空间划分为有限个单元。

CART 由两部分组成:

* **决策树生成**:生成的决策树要尽量大

* **决策树剪枝**:用验证数据集对已生成的树进行剪枝并选择最优树，此时使用损失函数作为剪枝的标准。


### CART 生成

回归树使用平方误差最小化准则，对分类树用基尼指数最小化准则，进行特征选择，生成二叉树。

#### 回归树生成

一个回归树对应着输入空间的一个划分以及在划分的单元上的输出值，假设已将输入空间划分为 M 个单元 R_1,R_2,...,R_M, 并且在每个单元 R_m 上有一个固定输出值 c_m，于是回归树模型可表示为:

$$f(x)=\sum^M_{m=1}c_mI(x\in R_m)$$

当输入空间确定时，使用平方误差

$$\sum_{x_i \in R_m}(y_i - f(x_i))^2$$

表示回归树对于训练数据的预测误差，用平方误差最小准则求解每个单元上的最优输出值。

即最优值 c_m 是 R_m 上所有输入实例 x_i 对应的输出 y_i 的均值，即

$$\hat{e_m}=ave(y_i | x_i \in R_m)$$

选择第 j 个变量 x^{(j)} 和它的取值 s ， 作为切分变量和切分点，并定义两个区域:

$$R_1(j,s)=\{x|x^{(j)} \le s\}$$

$$ R_2(j,s)=\{x|x^{(j)} > s\}$$

然后寻找最佳切分变量 j 和最优切分点:

$$min_{j,s}[min_{c_1}\sum_{x \in R_1(j,s)}(y_i - c_i)^2 + min_{c_2}\sum_{x \in R_2(j,s)}(y_i - c_i)^2]$$

求解可得:

$$\hat{c_1}=ave(y_i | x_i \in R_1(j,s))$$

$$\hat{c_2}=ave(y_i | x_i \in R_2(j,s))$$

遍历所有输入变量，将输入空间划为两个区域。接着对两个区域重复上述划分过程，直到满足停止条件为止，这样就生成一棵回归树，通常称为最小二乘回归树。

#### 最小二乘回归树生成算法

#### 分类树生成


#### CART 生成算法

### CART 剪枝